In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

from nltk.corpus import stopwords
from wordcloud import WordCloud

In [6]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

In [56]:
# read one of json file for test purpose"C:/Users/Diriba/Desktop/10AC/Projects/week0_starter_network_analysis/anonymized/
data = pd.read_json("../users.json")

FileNotFoundError: File users.json does not exist

In [42]:
# printing 5 rows from the top
data.head()

,evariste,asalawu,yabebal,mmusa,admin,amanuelzewdu21,narega678,henokdes1,janerosenyams,birtukankuma1113,...,ehirwa,tigisthay13,isaaclucky88,mohammedesam.mem,maggycheppy,azaria,google_calendar,canva,googledrive,polly
id,U03T89ACUUW,U03TEPYRM2P,U03TNP8Q8CT,U03TT5KEYCF,U03TX2VN6H5,U03U1FNPEUX,U03U1FQKEMV,U03U1GHT39V,U03U1HAG9TR,U03U1J51VFZ,...,U03V1AM5TFA,U03V5Q9N516,U03V61VGQG0,U03V6HMRPGQ,U03V785NLSU,U03V8LHPDME,U03VAH809FC,U0401GU4W6L,U0418T6CD1B,U04718Y7SQ0
first_name,Carlos,Garrett,Bethany,Luis,Nicole,Joshua,Steven,Joseph,Robert,Cheryl,...,Vanessa,Debra,Paula,Cristian,Ashley,Mandy,Paul,Kathleen,Kenneth,Audrey
last_name,Gross,Bell,George,Ingram,Kim,Rhodes,Garcia,Diaz,Carter,Hudson,...,Norman,Lawrence,Bryant,Wilson,Hamilton,Sweeney,Brown,Phillips,Fitzgerald,Parsons
name,Carlos,Garrett,Bethany,Luis,Nicole,Joshua,Steven,Joseph,Robert,Cheryl,...,Vanessa,Debra,Paula,Cristian,Ashley,Mandy,Paul,Kathleen,Kenneth,Audrey
email,Carlos@10academy.org,Garrett@10academy.org,Bethany@10academy.org,Luis@10academy.org,Nicole@10academy.org,Joshua@10academy.org,Steven@10academy.org,Joseph@10academy.org,Robert@10academy.org,Cheryl@10academy.org,...,Vanessa@10academy.org,Debra@10academy.org,Paula@10academy.org,Cristian@10academy.org,Ashley@10academy.org,Mandy@10academy.org,Paul@10academy.org,Kathleen@10academy.org,Kenneth@10academy.org,Audrey@10academy.org


In [37]:
# printing 5 items from the bottom
data.tail()

,id,team_id,name,deleted,color,real_name,tz,tz_label,tz_offset,profile,...,is_owner,is_primary_owner,is_restricted,is_ultra_restricted,is_bot,is_app_user,updated,is_email_confirmed,who_can_share_contact_card,is_invited_user
66,U03V8LHPDME,T03U4J8HMUG,Mandy,False,e96699,Mandy Sweeney,America/New_York,Eastern Standard Time,-18000.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,False,False,1667364708,1.0,EVERYONE,NaN
67,U03VAH809FC,T03U4J8HMUG,Paul,False,df3dc0,Paul Brown,America/Los_Angeles,Pacific Standard Time,-28800.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,True,False,1661515236,0.0,EVERYONE,NaN
68,U0401GU4W6L,T03U4J8HMUG,Kathleen,False,2b6836,Kathleen Phillips,America/Los_Angeles,Pacific Standard Time,-28800.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,True,False,1661252626,0.0,EVERYONE,NaN
69,U0418T6CD1B,T03U4J8HMUG,Kenneth,False,4cc091,Kenneth Fitzgerald,America/Los_Angeles,Pacific Standard Time,-28800.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,True,False,1662534866,0.0,EVERYONE,NaN
70,U04718Y7SQ0,T03U4J8HMUG,Audrey,False,9b3b45,Audrey Parsons,America/Los_Angeles,Pacific Standard Time,-28800.0,"{'title': '', 'phone': '', 'skype': '', 'real_...",...,0.0,0.0,0.0,0.0,True,False,1665649740,0.0,EVERYONE,NaN


In [43]:
# print desccription of the data 
data.describe()

,evariste,asalawu,yabebal,mmusa,admin,amanuelzewdu21,narega678,henokdes1,janerosenyams,birtukankuma1113,...,ehirwa,tigisthay13,isaaclucky88,mohammedesam.mem,maggycheppy,azaria,google_calendar,canva,googledrive,polly
count,9,9,9,9,9,9,9,9,9,9,...,9,9,9,9,9,9,9,9,9,9
unique,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
top,Carlos Gross,Garrett Bell,Bethany George,Luis Ingram,Nicole Kim,Joshua Rhodes,Steven Garcia,Joseph Diaz,Robert Carter,Cheryl Hudson,...,Vanessa Norman,Debra Lawrence,Paula Bryant,Cristian Wilson,Ashley Hamilton,Mandy Sweeney,Paul Brown,Kathleen Phillips,Kenneth Fitzgerald,Audrey Parsons
freq,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,4,4,4,4,4,4


In [44]:
# know the shape of the data 
data.shape

(9, 71)

In [45]:
# print name of the data coumns
data.columns

Index(['evariste', 'asalawu', 'yabebal', 'mmusa', 'admin', 'amanuelzewdu21',
       'narega678', 'henokdes1', 'janerosenyams', 'birtukankuma1113',
       'mkiprono', 'arun', 'nardos', 'birhanugebisa', 'egidekamuzinzi',
       'tibarekberassu', 'hnnadesta', 'softeng2006', 'gwangui00000',
       'tenxdapp', 'wsennay', 'andenet.dev', 'github', 'mahlet', 'tinsae',
       'eyuelseyoum92', 'yohanssamuel2014', 'zelalemshiferaw71921',
       'habtamuabate937', 'kbkibatu', 'mikygetyos', 'antenehtilaye19',
       'thamarniyo', 'haylemicheal.mekonnen', 'nahomhabtemichael',
       'dawitasmero', 'gdekebo2020', 'amdeamd7', 'josias.ouns.djossou',
       'hundekey3', 'yohgut', 'likye1823', 'natnaelmasresha', 'degagawolde',
       'axutec14', 'emtinan.s.e.osman', 'aojutomori', 'gsinseswa721',
       'hermelagetnet90', 'yididya', 'ashittu', 'akiiru', 'latseden2',
       'yastesh10', 'bekiman21', 'yabebal535', 'gedionandme', 'natrix277',
       'prubayita', 'fisseha.137', 'bereket', 'ehirwa', 'tigisthay

### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [ ]:
# which user has the highest number of reply counts?


In [ ]:
# Visualize reply counts per user per channel

In [ ]:
# what is the time range of the day that most messages are sent?


In [11]:
# what kind of messages are replied faster than others?

In [15]:
# Relationship between # of messages and # of reactions

In [16]:
# Classify messages into different categories such as questions, answers, comments, etc.

In [17]:
# Which users got the most reactions?

In [33]:
# Model topics mentioned in the channel

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?